<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Memento

## Intro

The memento captures and externalises an objects internal state so that the object can be restored to this state later. This should not at all violate encapsulation to achieve this (Shouldn't leak details or be usable outside of this context).

The aim of the game here is time travel. We should be able to navigate to previous state of an object easily.

A good example of a memento is CTRL+Z. The changelist can be rolled through backwards to return to a previous state of the application.

We need this pattern to implement checkpoints or to recover from errors in tentative operation contexts.

We need to save the state somewhere that is accessible to the object and nothing else. Objects normally encapsulate all of their own state making it innaccessible to other objects and impossible to save externally.

Advantages of the pattern are:
* Simple to implements
* Provides recovery capability to your program

The memento avoids exposing information only an originator (the thing having its state saved) should manage, but that must be stored outside the originator. It shields the objects from potentially complex internals thereby maintaining boundaries.

mementos can incur considerable cost when copying large amounts of information. The only real way to apply this pattern is if it is cheap to do so. If not, a structural pattern may be more effective.

## Implementation

In [23]:
class FileWriterUtil {
    private String filename;
    private StringBuilder content; // stand in for file read in
    
    public FileWriterUtil(String file)
    {
        this.filename = file;
        this.content = new StringBuilder();
    }
    
    // Overriding to string of class to simulate file read out
    @Override
    public String toString()
    {
        return this.content.toString();
    }
    
    public void write(String str)
    {
        content.append(str);
    }
    
    public Memento save()
    {
        return new Memento(this.filename, this.content);
    }
    
    public void undo(Object obj)
    {
        Memento mmt = (Memento) obj;
        this.filename = mmt.filename;
        this.content = mmt.content;
    }
    
    // Memento inner class. Best way to do this in Java really. 
    private class Memento {
        private String filename;
        private StringBuilder content;
        
        public Memento(String filename, StringBuilder content)
        {
            this.filename = filename;
            this.content = new StringBuilder(content);
        }
    }
}

In [24]:
class FileWriterCaretaker {
    private Object obj;
    
    public void save(FileWriterUtil fwu)
    {
        this.obj = fwu.save();
    }
    
    public void undo(FileWriterUtil fwu)
    {
        fwu.undo(obj);
    }
}

In [45]:
FileWriterCaretaker caretaker = new FileWriterCaretaker();

In [46]:
FileWriterUtil writer = new FileWriterUtil("test.txt");

In [47]:
writer.write("This is my 1st bit of data");

In [48]:
System.out.println(writer);

This is my 1st bit of data


In [49]:
caretaker.save(writer);

In [50]:
writer.write("\nEven more data...");

In [51]:
System.out.println(writer);

This is my 1st bit of data
Even more data...


In [52]:
caretaker.undo(writer);

In [53]:
System.out.println(writer);

This is my 1st bit of data


## Example

In [98]:
class Counter {
    private int count = 0;
    
    public Memento add()
    {
        count++;
        return new Memento(count);
    }
    
    public Memento subtract()
    {
        count--;
        return new Memento(count);
    }
    
    public void undo(Object obj)
    {
        Memento mmt = (Memento) obj;
        this.count = mmt.count;
    }
    
    public void show()
    {
        System.out.println(count);
    }
    
    private class Memento {
        private int count;
        
        public Memento(int num)
        {
            this.count = num;
        }
    }
}

In [99]:
class CountCaretaker {
    private ArrayList<Object> states = new ArrayList<>();
    
    public void add(Counter count)
    {
        states.add(count.add());
    }
    
    public void subtract(Counter count)
    {
        states.add(count.subtract());
    }
    
    public void undoLast(Counter count)
    {
        states.remove(states.size() - 1);
        count.undo(states.get(states.size() - 1));
    }
    
    public void undoToFirstState(Counter count)
    {
        count.undo(states.get(0));
    }
}

In [100]:
Counter counter = new Counter();
CountCaretaker myCare = new CountCaretaker();

In [101]:
myCare.add(counter);
myCare.add(counter);
myCare.add(counter);

counter.show();

3


In [102]:
counter.show();

3


In [103]:
myCare.undoLast(counter);
counter.show();

2


In [104]:
myCare.undoLast(counter);
counter.show();

1
